MIT License

Copyright (c) 2024 Mohannad Shehadeh

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
using LinearAlgebra

In [2]:
# Gauss–Jordan elimination to get RREF FOR GF(2)-VALUED MATRICES ONLY
function RREF(M)
    M = copy(M)
    local m = size(M)[1]
    local n = size(M)[2]
    local h = 1
    local k = 1
    while h <= m && k <= n
        pivRow = findnext(M[:,k] .!= false, h)
        if pivRow == nothing
            k += 1            
        else
            swapRows!(M, h, pivRow)
            for i = h + 1 : m
                if M[i,k] == true
                    M[i,:] = M[i,:] .⊻ M[h,:]
                end
            end
            # Remove to get REF instead of RREF
            for i = 1 : h - 1 
                if M[i,k] == true
                    M[i,:] = M[i,:] .⊻ M[h,:]
                end
            end
            h += 1; k += 1
        end
    end
    return M
end

function swapRows!(M, i, j)
    tempCopy = M[i,:]
    M[i,:] = M[j,:]
    M[j,:] = tempCopy
    return M
end

swapRows! (generic function with 1 method)

In [3]:
# Get generator matrix of DUAL CODE of code generated by G FOR GF(2)-VALUED MATRICES ONLY
function dual(G)
    A = RREF(G)
    local k = size(G)[1]
    local n = size(G)[2]
    info_positions = []
    for i in 1:k
        position = findfirst(x->x!=0,A[i,:])
        if position == nothing
            break
        else
            push!(info_positions, position)
        end
    end
    k = length(info_positions)
    parity_positions = [i for i in 1:n if i ∉ info_positions]
    dual_G = zeros(Int64, n-k, n)
    G_parity_view = view(A[1:k,:],:,parity_positions)
    H_parity_view = view(dual_G,:,parity_positions)
    H_info_view = view(dual_G,:,info_positions)
    copyto!(H_info_view,transpose(G_parity_view))  
    vertical_offset = 0
    for i in parity_positions
        vertical_offset += 1
        dual_G[vertical_offset, i] = 1
    end
    return dual_G
end

dual (generic function with 1 method)

In [4]:
# extending Hamming code parameter (code length = 2^m)
# n = rand(2^3:2^16)
n = 2^6
m = ceil(Int64,log2(n))
s = 2^m-n
println("n $n")
println("m $m")
println("s $s")

n 64
m 6
s 0


Affine-transformation-based:

In [5]:
# m a b ainv
mabainv = 
[
3 1 1 1;  
4 3 0  11;
5 3 0  11;
6 3 3  43;
7 5 5  77;
8 9 11  57;
9 19 19  27;
10 27 27  531;
11 53 53  541;
12 89 89  2025;
13 163 170  4875;
14 301 308  13989;
15 553 553  14873;
16 1065 1155 55321 
]

a = mabainv[m-3+1,2]
b = mabainv[m-3+1,3] + a*s
ainv = mabainv[m-3+1,4]
mask = (1<<m)-1
mb = (1<<m)-b

# forward and inverse permutations 
# left/right shifts are multiplication/division by 2
# bitwise ANDing with mask is modulo 2^m_ reduction
# these operations can also simply be done with normal * and mod(,2)
h(i) = (((a*i+b)&mask)<<1)+1
p(h) = ((ainv*((h>>1)+mb))&mask)

H_special_1 = zeros(Int64,m+1,0)
H_conventional = zeros(Int64,m+1,0)
for i in 0:n-1
    H_special_1 = hcat(H_special_1,reverse(digits(h(i),base=2,pad=m+1))) 
    H_conventional = hcat(H_conventional,reverse(digits(2*i+1,base=2,pad=m+1))) 
end

@assert [p(h(x)) for x in 0:n-1] == 0:n-1
@assert [h(p(2*x+1)) for x in 0:n-1] == 2 .* (0:n-1) .+ 1

Boolean-function-based:

In [6]:
ind_deg_sig = 
[
0 0 0;
1 0 0;
2 2 0;
3 2 0;
4 3 0;
5 3 1;
6 3 2;
7 2 0;
8 4 0;
9 4 1;
10 4 2;
11 4 3;
12 4 4;
13 4 5;
14 4 6;
15 3 0;
]

θ(s,d,x) = d == 0 ? 0 : prod([((s>>j)&0x1)⊻((x>>j)&0x1) for j in 0:d-1])
function ff(i)
    result = 0
    for bit in 0:m-1
        degree = ind_deg_sig[bit+1,2]
        signature = ind_deg_sig[bit+1,3]
        result = result ⊻ ((((i>>bit)&0x1) ⊻ θ(signature,degree,i)) << bit)
    end
    return 2*result + 1
end

function gg(h)
    local i = h>>1
    result = 0
    for bit in 0:m-1
        degree = ind_deg_sig[bit+1,2]
        signature = ind_deg_sig[bit+1,3]
        result = result ⊻ ((((i>>bit)&0x1) ⊻ θ(signature,degree,result)) << bit)
    end
    return result
end

@assert [gg(ff(x+s)) for x in 0:n-1] == (0:n-1) .+ s
@assert [ff(gg(2*(x+s)+1)) for x in 0:n-1] == 2 .* ((0:n-1) .+ s) .+ 1

H_special_2 = zeros(Int64,m+1,0)
for i in 0:n-1
    H_special_2 = hcat(H_special_2,reverse(digits(ff(i+s),base=2,pad=m+1))) 
end

In [7]:
# H_conventional

In [8]:
# G_conventional = dual(H_conventional)

In [9]:
# G_conventional_rref = RREF(G_conventional)

In [10]:
# Are the two Gs true generator matrices? SHOULD BE THE CASE!
# Both of the following matrix products should be all zero

In [11]:
# mod.(G_conventional*transpose(H_conventional),2)

In [12]:
# mod.(G_conventional_rref*transpose(H_conventional),2)

In [13]:
# Let's now try the special one

In [14]:
H_special_1

7×64 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  1  1  1  …  0  1  1  1  1  1  1  1  1  1  1  0
 0  0  0  0  0  1  1  1  1  1  0  0  0     1  0  0  0  0  0  1  1  1  1  1  0
 0  0  1  1  1  0  0  1  1  1  0  0  0     1  0  0  1  1  1  0  0  0  1  1  0
 0  1  0  1  1  0  1  0  0  1  0  1  1     1  0  1  0  0  1  0  1  1  0  1  0
 1  1  0  0  1  1  0  0  1  1  0  0  1     1  1  0  0  1  1  0  0  1  1  0  0
 1  0  1  0  1  0  1  0  1  0  1  0  1  …  1  0  1  0  1  0  1  0  1  0  1  0
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1

In [15]:
# G_special = dual(H_special_1);
# G_special_rref = RREF(G_special); # IS SYSTEMATIC!!! 
# k = n-(m+1)
# Id_mat = Int64.(ones(k,k)^0)
# println(G_special_rref[1:k,1:k] == Id_mat)
# println(sum(mod.(G_special_rref*transpose(H_special_1),2)) == 0)

mod(det(BigInt.(H_special_1[:,end-(m+1)+1:end])),2)

1

In [16]:
H_special_2

7×64 Matrix{Int64}:
 0  0  0  0  0  0  1  0  0  0  0  0  0  …  1  1  0  1  1  1  1  1  1  1  0  1
 0  0  0  0  0  0  0  1  0  0  0  0  0     1  1  1  0  1  1  1  1  1  1  1  0
 0  0  0  1  0  0  0  1  1  1  1  0  1     0  0  0  1  1  1  1  0  1  1  1  0
 0  0  0  1  1  1  1  0  0  0  0  1  1     1  1  1  0  0  0  0  1  1  1  1  0
 0  0  1  1  0  0  1  1  0  0  1  1  0     0  0  1  1  0  0  1  1  0  0  1  1
 0  1  0  1  0  1  0  1  0  1  0  1  0  …  0  1  0  1  0  1  0  1  0  1  0  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1

In [17]:
# G_special = dual(H_special_2);
# G_special_rref = RREF(G_special); # IS SYSTEMATIC!!! 
# k = n-(m+1)
# Id_mat = Int64.(ones(k,k)^0)
# println(G_special_rref[1:k,1:k] == Id_mat)
# println(sum(mod.(G_special_rref*transpose(H_special_2),2)) == 0)

mod(det(BigInt.(H_special_2[:,end-(m+1)+1:end])),2)

1

In [18]:
H_special_2[:,end-7+1:end]

7×7 Matrix{Int64}:
 1  1  1  1  1  0  1
 1  1  1  1  1  1  0
 1  1  0  1  1  1  0
 0  0  1  1  1  1  0
 0  1  1  0  0  1  1
 1  0  1  0  1  0  1
 1  1  1  1  1  1  1

In [19]:
int_rep = ff.([57,58,59,60,61,62,63])

7-element Vector{Int64}:
 115
 117
 111
 121
 123
  61
  71

In [20]:
reverse.(digits.(int_rep,base=2,pad=m+1))

7-element Vector{Vector{Int64}}:
 [1, 1, 1, 0, 0, 1, 1]
 [1, 1, 1, 0, 1, 0, 1]
 [1, 1, 0, 1, 1, 1, 1]
 [1, 1, 1, 1, 0, 0, 1]
 [1, 1, 1, 1, 0, 1, 1]
 [0, 1, 1, 1, 1, 0, 1]
 [1, 0, 0, 0, 1, 1, 1]

In [21]:
det(H_special_2[:,end-7+1:end])

-1.0